<a href="https://colab.research.google.com/github/dpk-a7/Information-Retrieval-IR-/blob/main/Document_search1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

https://iq.opengenus.org/document-similarity-tf-idf/

In [ ]:
import re
import nltk
import math
!pip install contractions
import contractions
import numpy as np
from nltk.stem import PorterStemmer
from nltk.stem import LancasterStemmer
from nltk.stem import WordNetLemmatizer
from nltk.stem.snowball import SnowballStemmer
from nltk.tokenize import word_tokenize
nltk.download('wordnet')
nltk.download('punkt')
from sklearn.feature_extraction.text import TfidfVectorizer,CountVectorizer
import pandas as pd

     |████████████████████████████████| 321 kB 5.3 MB/s 
     |████████████████████████████████| 284 kB 43.3 MB/s 
  Created wheel for pyahocorasick: filename=pyahocorasick-1.4.2-cp37-cp37m-linux_x86_64.whl size=85452 sha256=3529cc4f4a9d1eb2c7015f82ca2916649aa9cd7eeafda98f0daa81a3d8cd7b13
  Stored in directory: /root/.cache/pip/wheels/25/19/a6/8f363d9939162782bb8439d886469756271abc01f76fbd790f
Successfully built pyahocorasick
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


Sentence ->  Decontractor ->  Remove Punctuations -> Tokenization & Extra Spaces -> Normalization(Lemmatization or Stemming) -> Cpython(TF-IDF) !-> Term-based Similarity measuer


In [ ]:
# Sentence
X = ["HC:Unsafe condition: Access not provided in the excavation for ingress/egress",
     "I am interested in this meet"][1]
Y = ["HC:Unsafe condition: Excavation access not provided",
     "HC:Unsafe environment: No access for is provided in the excavation",
     "HC:Unsafe condition: Entry and exit is not provided",
     "HC:Unsafe condition: No access for ingress/egress is provided in the excavation",
     "I not at all interested in this meet",
     "I am kind of interested in this meet"][5]

In [ ]:
from difflib import SequenceMatcher

def similar(a, b):
    return SequenceMatcher(None, a, b).ratio()
print(similar(X,Y))

0.875


In [ ]:
# Decontraction solution 1
def decontracted(phrase):
    # specific
    phrase = re.sub(r"won\'t", "will not", phrase)
    phrase = re.sub(r"can\'t", "can not", phrase)
    phrase = re.sub(r"wouldn\'t", "would not", phrase)
    phrase = re.sub(r"wouldn\'t", "would not", phrase)
    # general
    phrase = re.sub(r"an\'t", "am not", phrase)
    phrase = re.sub(r"n\'t", " not", phrase)
    phrase = re.sub(r"\'re", " are", phrase)
    phrase = re.sub(r"\'s", " is", phrase)
    phrase = re.sub(r"\'d", " would", phrase)
    phrase = re.sub(r"\'ll", " will", phrase)
    phrase = re.sub(r"\'t", " not", phrase)
    phrase = re.sub(r"\'ve", " have", phrase)
    phrase = re.sub(r"\'m", " am", phrase)
    return phrase

In [ ]:
# Decontraction solution 2
def deconstration(sentS:str)-> "string":
  # creating an empty list
  expanded_words = []    
  for word in sentS.split(" "):
    expanded_words.append(contractions.fix(word))       
  return ' '.join(expanded_words)
X = deconstration(X)
Y = deconstration(Y)
print(f"X decontracted: {X} \nY decontracted: {Y}")

X decontracted: I am interested in this meet 
Y decontracted: I am kind of interested in this meet


In [ ]:
X = re.sub(r'[^\w\s]', '', X)
Y = re.sub(r'[^\w\s]', '', Y)
print(f"X removed Punctuations: {X} \nY removed Punctuations: {Y}")

X removed Punctuations: I am interested in this meet 
Y removed Punctuations: I am kind of interested in this meet


In [ ]:
# Tokenization and  Extra Spaces remove
X_t = word_tokenize(X)
Y_t = word_tokenize(Y)
print(f"X tokenized: {X_t} \nY tokenized: {Y_t}")

X tokenized: ['I', 'am', 'interested', 'in', 'this', 'meet'] 
Y tokenized: ['I', 'am', 'kind', 'of', 'interested', 'in', 'this', 'meet']


In [ ]:
# Normalization(Lemmatization or Stemming)
porter = PorterStemmer()
lancaster=LancasterStemmer()
wordnet_lemmatizer = WordNetLemmatizer()
englishStemmer=SnowballStemmer("english")

port_X  = lambda i: [porter.stem(w) for w in i]
lanc_X  = lambda i: [lancaster.stem(w) for w in i]
snow_X  = lambda i: [englishStemmer.stem(w) for w in i]
lemma_X = lambda i: [wordnet_lemmatizer.lemmatize(w, pos="v") for w in i]

In [ ]:
print("PorterStemmer: ", port_X(X_t))
print("LancasterStemmer: ", lanc_X(X_t)) #<- fast and simple
print("SnowballStemmer: ", snow_X(X_t))
print("WordNetLemmatizer:", lemma_X(X_t)) #<- slow and efficient

PorterStemmer:  ['I', 'am', 'interest', 'in', 'thi', 'meet']
LancasterStemmer:  ['i', 'am', 'interest', 'in', 'thi', 'meet']
SnowballStemmer:  ['i', 'am', 'interest', 'in', 'this', 'meet']
WordNetLemmatizer: ['I', 'be', 'interest', 'in', 'this', 'meet']


In [ ]:
X_norm = lanc_X(X_t)
Y_norm = lanc_X(Y_t)
print(f"X Normalized: {X_norm} \nY Normalized: {Y_norm}")

X Normalized: ['i', 'am', 'interest', 'in', 'thi', 'meet'] 
Y Normalized: ['i', 'am', 'kind', 'of', 'interest', 'in', 'thi', 'meet']


In [ ]:
countvectorizer = CountVectorizer(analyzer= 'word')
tfidfvectorizer = TfidfVectorizer(analyzer='word')

train = [" ".join(X_norm), " ".join(Y_norm)]
count_wm = countvectorizer.fit_transform(train)
tfidf_wm = tfidfvectorizer.fit_transform(train)

count_tokens = countvectorizer.get_feature_names()
tfidf_tokens = tfidfvectorizer.get_feature_names()

df_countvect = pd.DataFrame(data = count_wm.toarray(),index = ['Doc1','Doc2'],columns = count_tokens)
df_tfidfvect = pd.DataFrame(data = tfidf_wm.toarray(),index = ['Doc1','Doc2'],columns = tfidf_tokens)
print("Count Vectorizer\n")
print(df_countvect)
print("\nTD-IDF Vectorizer\n")
print(df_tfidfvect)

Count Vectorizer

      am  in  interest  kind  meet  of  thi
Doc1   1   1         1     0     1   0    1
Doc2   1   1         1     1     1   1    1

TD-IDF Vectorizer

            am        in  interest      kind      meet        of       thi
Doc1  0.447214  0.447214  0.447214  0.000000  0.447214  0.000000  0.447214
Doc2  0.334251  0.334251  0.334251  0.469778  0.334251  0.469778  0.334251


# Cosine similarity

In [ ]:
l1 = tfidf_wm.toarray()[0]
l2 = tfidf_wm.toarray()[1]
c = 0
for i_1 in range(len(tfidf_tokens)):
  c += l1[i_1] * l2[i_1]
cosine = c / float((sum(l1) * sum(l2)) ** 0.5)
print("Cosine using tfidf",cosine)

Cosine using tfidf 0.3093337485597601


In [ ]:
l1 = count_wm.toarray()[0]
l2 = count_wm.toarray()[1]
c = 0
for i_1 in range(len(tfidf_tokens)):
  c += l1[i_1] * l2[i_1]
cosine = c / float((sum(l1) * sum(l2)) ** 0.5)
print("Cosine using count vectorizer",cosine)

Cosine using count vectorizer 0.8451542547285166


# Jaccard Distance

In [ ]:
 # Jaccard Similarity(d1, d2) = d1 ∩ d2 / d1 ∪ d2
JD = len(set(tfidf_wm.toarray()[0]).intersection(set(tfidf_wm.toarray()[1])))/len(set(tfidf_wm.toarray()[0]).union(set(tfidf_wm.toarray()[1])))
print("Jaccard similarity using TFIDF: ",JD)

Jaccard similarity using TFIDF:  0.0


In [ ]:
def jaccard_binary(x,y):
    """A function for finding the similarity between two binary vectors"""
    intersection = np.logical_and(x, y)
    union = np.logical_or(x, y)
    similarity = intersection.sum() / float(union.sum())
    return similarity
print("Jaccard similarity using TFIDF: ", jaccard_binary(tfidf_wm.toarray()[0],tfidf_wm.toarray()[1]))
print("Jaccard similarity using Count vectorizer: ", jaccard_binary(count_wm.toarray()[0],count_wm.toarray()[1]))

Jaccard similarity using TFIDF:  0.7142857142857143
Jaccard similarity using Count vectorizer:  0.7142857142857143


# Euclidean Distance

In [ ]:
# Euclidean Distance = sqrt(∑(xi−yi)^2), where i = 1 to i = n (number of vectors)
dist1 = np.linalg.norm(tfidf_wm.toarray()[0] - tfidf_wm.toarray()[1])
print("Euclidean Distance using TFIDF:",dist1)
dist2 = np.linalg.norm(count_wm.toarray()[0] - count_wm.toarray()[1])
print("Euclidean Distance using Count vectorizer:",dist2)

Euclidean Distance using TFIDF: 0.7107638792087759
Euclidean Distance using Count vectorizer: 1.4142135623730951


In [ ]:

bool(dic)

False

In [ ]:
def deconstration(sentS:str)->"string":
  expanded_words = []
  for word in sentS.split(" "):
      expanded_words.append(contractions.fix(word))
  return ' '.join(expanded_words)
fileDict = dict()
dict_f = dict()
x = 0

with open("test1.txt") as file:
    temp = []
    for  f in file.readlines():
        temp.append(f.replace("\n", ''))
    temp.remove('')
    fileDict = {i: deconstration(v) for i, v in enumerate(temp)}
print(fileDict)

{0: 'HC:Unsafe condition: Excavation access not provided', 1: 'HC:Unsafe environment: No access for is provided in the excavation', 2: 'HC:Unsafe condition: Entry and exit is not provided', 3: 'HC:Unsafe condition: No access for ingress/egress is provided in the excavation'}


In [ ]:
# https://nlpforhackers.io/wordnet-sentence-similarity/
rom nltk import word_tokenize, pos_tag
nltk.download('averaged_perceptron_tagger')
from nltk.corpus import wordnet as wn
 
def penn_to_wn(tag):
    """ Convert between a Penn Treebank tag to a simplified Wordnet tag """
    if tag.startswith('N'):
        return 'n'
 
    if tag.startswith('V'):
        return 'v'
 
    if tag.startswith('J'):
        return 'a'
 
    if tag.startswith('R'):
        return 'r'
 
    return None
 
def tagged_to_synset(word, tag):
    wn_tag = penn_to_wn(tag)
    if wn_tag is None:
        return None
    try:
        return wn.synsets(word, wn_tag)[0]
    except:
        return None
def sentence_similarity(sentence1, sentence2):
    """ compute the sentence similarity using Wordnet """
    # Tokenize and tag
    sentence1 = pos_tag(word_tokenize(sentence1))
    sentence2 = pos_tag(word_tokenize(sentence2))
 
    # Get the synsets for the tagged words
    synsets1 = [tagged_to_synset(*tagged_word) for tagged_word in sentence1]
    synsets2 = [tagged_to_synset(*tagged_word) for tagged_word in sentence2]
 
    # Filter out the Nones
    synsets1 = [ss for ss in synsets1 if ss]
    synsets2 = [ss for ss in synsets2 if ss]
    print(synsets1)
    print(synsets2)
    score, count = 0.0, 0
 
    # For each word in the first sentence
    for synset in synsets1:
    # Get the similarity value of the most similar word in the other sentence
      simlist = [synset.path_similarity(ss) for ss in synsets2 if synset.path_similarity(ss) is not None]
      if not simlist:
          continue;
      best_score = max(simlist)

      # Check that the similarity could have been computed
      score += best_score
      count += 1

    if count == 0:
      return 0

    # Average the values
    score /= count
    return score

def symmetric_sentence_similarity(sentence1, sentence2):
    """ compute the symmetric sentence similarity using Wordnet """
    return (sentence_similarity(sentence1, sentence2) + sentence_similarity(sentence2, sentence1)) / 2 
 
sentences = [
    "HC:Unsafe condition: Excavation access not provided",
    "HC:Unsafe environment: No access for is provided in the excavation",
    "HC:Unsafe condition: Entry and exit is not provided",
    "HC:Unsafe condition: No access for ingress/egress is provided in the excavation",
]
 
focus_sentence = "HC:Unsafe condition: Access provided in the excavation for ingress/egress"
 
for sentence in sentences:
    print("Similarity(\"%s\", \"%s\") = %s" % (focus_sentence, sentence, symmetric_sentence_similarity(focus_sentence, sentence)))
    # print("Similarity(\"%s\", \"%s\") = %s" % (sentence, focus_sentence, symmetric_sentence_similarity(sentence, focus_sentence)))
    print()

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[Synset('insecure.a.02'), Synset('condition.n.01'), Synset('entree.n.02'), Synset('supply.v.01'), Synset('excavation.n.01')]
[Synset('insecure.a.02'), Synset('condition.n.01'), Synset('excavation.n.01'), Synset('entree.n.02'), Synset('not.r.01'), Synset('supply.v.01')]
[Synset('insecure.a.02'), Synset('condition.n.01'), Synset('excavation.n.01'), Synset('entree.n.02'), Synset('not.r.01'), Synset('supply.v.01')]
[Synset('insecure.a.02'), Synset('condition.n.01'), Synset('entree.n.02'), Synset('supply.v.01'), Synset('excavation.n.01')]
Similarity("HC:Unsafe condition: Access provided in the excavation for ingress/egress", "HC:Unsafe condition: Excavation access not provided") = 1.0

[Synset('insecure.a.02'), Synset('condition.n.01'), Synset('entree.n.02'), Synset('supply.v.01'), Synset('excavation.n.01'